# Download historical weather data:
## Inspect the webpage:  
http://www.weather.gov.sg/climate-historical-daily/

Import the following modules:-
 * BeautifulSoup  
 * urllib.request  
 * webdriver from selenium  
 * re  

In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request as r
from selenium import webdriver
import re

Set url:

In [ ]:
url = "http://www.weather.gov.sg/climate-historical-daily/"

Set webdriver:

In [ ]:
driver = webdriver.Firefox(executable_path='../../../WebDriver/geckodriver')

Retrieve page:

In [ ]:
driver.get(url)

In [ ]:
html = driver.page_source

Gather BeautifulSoup object:

In [ ]:
bsObj = bs(html,"html5lib")

Find all links on page:

In [ ]:
bsObj.find_all('a')

## Download a csv dataset
Set opener:

In [ ]:
opener = r.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]

Create output directory:

In [ ]:
import os
out_path = "../../../data/1_raw/SG_weather/"
if not os.path.exists(out_path):
    os.makedirs(out_path)

Retrieve link from BeautifulSoup object:

In [ ]:
link = bsObj.find('a', text="CSV")['href']
link

Get filename:

In [ ]:
from urllib.parse import urlsplit
urlsplit(link)

In [ ]:
filename = os.path.basename(urlsplit(link).path)
filename

Download the file from and save it:

In [ ]:
with opener.open(link) as response, open(out_path+filename, 'wb') as out_file:
    data = response.read() # a `bytes` object
    out_file.write(data)

## Retrieve all historical data
Examining the website, we will find the best way to retrieve all data:
 * loop through all the weather stations
 * then loop through the years
 * followed by the months
 * finally retrieve the CSV file links for that month of year of the station

Get list of cityname:

In [ ]:
list_cityname = bsObj.find('button',id="cityname").find_next_sibling()

Loop to retrieve weather stations:

In [ ]:
citynames = []
for item in list_cityname.children:
    if not item.isspace:
        cityname = item.find('a').text
        citynames.append(cityname)

In [ ]:
citynames

Find button ids for month and year:

In [ ]:
# similarly set button id for month and year
id_month = 'month'
id_year = 'year'

import time module to let us wait to allow for page to load.

In [ ]:
import time

Set driver:

In [ ]:
driver = webdriver.Firefox(executable_path='../../../WebDriver/geckodriver')
driver.get(url)

Set function to return html page after each click:

In [ ]:
def get_html(id_str, link_text):
    driver.find_element_by_id(id_str).click()
    driver.find_element_by_link_text(link_text).click()
    time.sleep(1)
    if id_str==id_month :
        driver.find_element_by_id('display').click()
        time.sleep(2)
    return driver.page_source

Create loop to retrieve links of all historical weather data:

In [ ]:
valid_months = ['December',
               'November',
               'October',
               'September',
               'August',
               'July',
               'June',
               'May',
               'April',
               'March',
               'February',
               'January']

In [ ]:
file_links = []
for cityname in citynames:
    html_city = get_html('cityname', cityname)
    bs_city = bs(html_city,"lxml")
    list_year = bs_city.find('button',id=id_year).find_next_sibling()
    
    years = []
    for item in list_year.children:
        if not item.isspace:
            year = item.find('a').text
            years.append(year)
    
    for year in years:
        html_year = get_html(id_year, year)
        bs_year = bs(html_year,"lxml")
        list_month = bs_year.find('button',id=id_month).find_next_sibling()
        for item in list_month.children:
            if not item.isspace:
                month = item.find('a').text
                if month in valid_months:
                    html_month = get_html(id_month, month)
                    bs_month = bs(html_month,"lxml")
                    link = bs_month.find('a', text="CSV")['href']
                    if link not in file_links:
                        file_links.append(link)
                        print(link)
    

Using links to retrieve all csv files:

In [ ]:
file_links[:3]

In [ ]:
for link in file_links:
    filename = os.path.basename(urlsplit(link).path)
    with opener.open(link) as response, open(out_path+filename, 'wb') as out_file:
        data = response.read() # a `bytes` object
        out_file.write(data)